# Toronto RE Capstone Project

###### This Notebook will be mainly used for the capstone project

In [2]:
import pandas as pd
import numpy as np
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


# Assignment for week 3:

In [74]:
!pip install folium
!pip install geopy
!pip install beautifulsoup4
!pip install geocoder

## Import libraries

In [85]:
import pandas as pd
import numpy as np
import requests


## download HTML data and read into dataframe

In [86]:
wiki_table = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df_data = wiki_table[0]
df_data.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## Filter data 

In [87]:
df_data=df_data[df_data['Borough']!='Not assigned']
df_data = df_data.reset_index(drop=True)
df_data.shape

(103, 3)

## Geocoder is not working properly, so will be using the file provided

In [88]:
url="http://cocl.us/Geospatial_data"
c=pd.read_csv(url)
c

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


## Join the 2 tables

In [89]:
df_data = df_data.merge(c, how='left',left_on='Postal Code', right_on='Postal Code')
df_data

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


## Now, some analysis with the data

In [42]:
# Exploratory analysis 
from geopy.geocoders import Nominatim 
import folium # map rendering library
import json

print('The dataframe has {} postal codes and {} boroughs'.format(
        len(df_data['Postal Code'].unique()),
        len(df_data['Borough'].unique())))


The dataframe has 103 postal codes and 10 boroughs


In [43]:
# get latitude and longitude of Toronto
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent='toronto')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of Toronto is {}, {}'.format(latitude, longitude))

The geographical coordinate of Toronto is 43.6534817, -79.3839347


In [44]:
# create a map of Toronto
map_toronto = folium.Map(location=[latitude,longitude], zoom_start=12)
# add markers to map

for lat, lng, borough, neighborhood in zip(df_data['Latitude'], df_data['Longitude'], df_data['Borough'], df_data['Neighbourhood']):
        label = '{}, {}'.format(neighborhood, borough)
        
        folium.CircleMarker(
            [lat,lng],
            radius =5,
            popup = label,
            color = 'blue',
            fill = True).add_to(map_toronto)
map_toronto

## Try using Foursquare API on this

In [45]:
CLIENT_ID = 'AWKEEWMDKNAWJBUM2OLPLVSCBJDKAEHSQKINKOO3N2GPWMI0' # your Foursquare ID
CLIENT_SECRET = 'HH20YAAJBL0FFBQWEQAJSW4DTPYGQ40ZWIEGFB34A4RKMLXG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AWKEEWMDKNAWJBUM2OLPLVSCBJDKAEHSQKINKOO3N2GPWMI0
CLIENT_SECRET:HH20YAAJBL0FFBQWEQAJSW4DTPYGQ40ZWIEGFB34A4RKMLXG


In [46]:
# explore Downtown
neighborhood_latitude = df_data.loc[2, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_data.loc[2, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_data.loc[2, 'Borough'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 

                                                               neighborhood_longitude))
LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        neighborhood_latitude,
        neighborhood_longitude,
        radius,
        LIMIT)
url


Latitude and longitude values of Downtown Toronto are 43.6542599, -79.3606359.


'https://api.foursquare.com/v2/venues/explore?client_id=AWKEEWMDKNAWJBUM2OLPLVSCBJDKAEHSQKINKOO3N2GPWMI0&client_secret=HH20YAAJBL0FFBQWEQAJSW4DTPYGQ40ZWIEGFB34A4RKMLXG&v=20180605&ll=43.6542599,-79.3606359&radius=500'

In [49]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fad66423a1e82333e844daa'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 44,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
 

In [57]:
venues= results['response']['groups'][0]['items']

df_venues = pd.json_normalize(venues)
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
df_venues = df_venues.loc[:,filtered_columns]

df_venues.head()

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Roselle Desserts,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",43.653447,-79.362017
1,Tandem Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",43.653559,-79.361809
2,Cooper Koo Family YMCA,"[{'id': '52e81612bcbc57f1066b7a37', 'name': 'D...",43.653249,-79.358008
3,Body Blitz Spa East,"[{'id': '4bf58dd8d48988d1ed941735', 'name': 'S...",43.654735,-79.359874
4,Impact Kitchen,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",43.656369,-79.356980


In [65]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

df_venues['venue.categories'] = df_venues.apply(get_category_type, axis=1)

In [67]:
df_venues.head(2)

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809


## Create function to repeat the same process for all Boroughs in Toronto

In [83]:
def getNearbyVenues(names, latitudes, longitudes, radius = 500):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns=['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

In [94]:
toronto_venues = getNearbyVenues(names = df_data.head(20)['Neighbourhood'],
                                latitudes = df_data.head(20)['Latitude'],
                                longitudes = df_data.head(20)['Longitude']
                                )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches


In [96]:
print(toronto_venues.shape)
toronto_venues.head()

(358, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [97]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Don Mills,23,23,23,23,23,23
"Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood",8,8,8,8,8,8
"Garden District, Ryerson",100,100,100,100,100,100
Glencairn,5,5,5,5,5,5
"Guildwood, Morningside, West Hill",8,8,8,8,8,8
Humewood-Cedarvale,5,5,5,5,5,5
"Lawrence Manor, Lawrence Heights",13,13,13,13,13,13
"Malvern, Rouge",1,1,1,1,1,1
"Parkview Hill, Woodbine Gardens",11,11,11,11,11,11


In [98]:
print('There are {} unique categories'.format(len(toronto_venues['Venue Category'].unique())))

There are 138 unique categories


In [136]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']],prefix="", prefix_sep="")

toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']
column_location = list(toronto_onehot.columns).index('Neighborhood')

fixed_columns =  list(toronto_onehot.columns)
fixed_columns.remove('Neighborhood')

fixed_columns = ['Neighborhood']+ fixed_columns
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot

,Neighborhood,Accessories Store,American Restaurant,Antique Shop,Art Gallery,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,...,Thai Restaurant,Theater,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
354,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
355,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
356,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [138]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,American Restaurant,Antique Shop,Art Gallery,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,...,Thai Restaurant,Theater,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Don Mills,0.000000,0.000000,0.000000,0.043478,0.043478,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
1,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
2,"Garden District, Ryerson",0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.010000,0.010000,...,0.010000,0.020000,0.01,0.00,0.000000,0.01,0.010000,0.010000,0.000000,0.000000
3,Glencairn,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,...,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
4,"Guildwood, Morningside, West Hill",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,...,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
5,Humewood-Cedarvale,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.20,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
6,"Lawrence Manor, Lawrence Heights",0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.076923,0.000000,0.076923,0.000000
7,"Malvern, Rouge",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
8,"Parkview Hill, Woodbine Gardens",0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.090909,...,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
9,Parkwoods,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000


In [141]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [142]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()



,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Don Mills,Gym,Coffee Shop,Japanese Restaurant,Beer Store,Art Gallery,Asian Restaurant,Restaurant,Discount Store,Caribbean Restaurant,Italian Restaurant
1,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",Pharmacy,Beer Store,Pizza Place,Liquor Store,Shopping Plaza,Coffee Shop,Convenience Store,Café,Dim Sum Restaurant,Dessert Shop
2,"Garden District, Ryerson",Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Japanese Restaurant,Cosmetics Shop,Furniture / Home Store,Middle Eastern Restaurant,Hotel,Ramen Restaurant
3,Glencairn,Pizza Place,Park,Pub,Bakery,Japanese Restaurant,Curling Ice,Diner,Dim Sum Restaurant,Dessert Shop,Department Store
4,"Guildwood, Morningside, West Hill",Breakfast Spot,Rental Car Location,Medical Center,Electronics Store,Mexican Restaurant,Intersection,Bank,Restaurant,Dance Studio,Curling Ice


In [143]:
from sklearn.cluster import KMeans

kcluster = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood',1)

kmeans = KMeans(n_clusters=kcluster, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_

array([1, 1, 1, 1, 1, 1, 1, 2, 1, 4, 1, 1, 3, 1, 1, 1, 0, 1])

In [165]:
#neighborhoods_venues_sorted.insert(0,'Cluster Labels', kmeans.labels_)
toronto_merged = df_data

toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood', how='left')
toronto_merged = toronto_merged.head(20)
toronto_merged.drop(5, inplace = True)
toronto_merged

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,4.0,Park,Food & Drink Shop,Yoga Studio,Dance Studio,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store,Curling Ice
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Intersection,Coffee Shop,Hockey Arena,Pizza Place,Portuguese Restaurant,Comfort Food Restaurant,Comic Shop,Construction & Landscaping,Convenience Store,Discount Store
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1.0,Coffee Shop,Bakery,Park,Pub,Breakfast Spot,Theater,Café,Art Gallery,Event Space,Performing Arts Venue
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1.0,Clothing Store,Accessories Store,Boutique,Gift Shop,Furniture / Home Store,Event Space,Coffee Shop,Women's Store,Vietnamese Restaurant,BBQ Joint
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1.0,Coffee Shop,Yoga Studio,Mexican Restaurant,Diner,Sandwich Place,Restaurant,Café,Chinese Restaurant,Portuguese Restaurant,Fried Chicken Joint
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,2.0,Fast Food Restaurant,Yoga Studio,Dance Studio,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store,Curling Ice,Dog Run
7,M3B,North York,Don Mills,43.745906,-79.352188,1.0,Gym,Coffee Shop,Japanese Restaurant,Beer Store,Art Gallery,Asian Restaurant,Restaurant,Discount Store,Caribbean Restaurant,Italian Restaurant
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,1.0,Pizza Place,Gym / Fitness Center,Bank,Bus Line,Pet Store,Breakfast Spot,Intersection,Pharmacy,Gastropub,Athletics & Sports
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1.0,Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Japanese Restaurant,Cosmetics Shop,Furniture / Home Store,Middle Eastern Restaurant,Hotel,Ramen Restaurant
10,M6B,North York,Glencairn,43.709577,-79.445073,1.0,Pizza Place,Park,Pub,Bakery,Japanese Restaurant,Curling Ice,Diner,Dim Sum Restaurant,Dessert Shop,Department Store


In [166]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kcluster)
ys = [i + x + (i*x)**2 for i in range(kcluster)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters